In [2]:
import boto3
import json
from jinja2 import Environment, FileSystemLoader
import sys
from pathlib import Path
import os

In [22]:
template_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","prompts-templates"))
output_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","model-results"))
ontology_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","ontologies"))
ontology_name = "PropertyAppraisalOntology-v2.ttl"
input_text_dir = os.path.abspath(os.path.join(os.getcwd(),"../..","prompts"))
input_text_name = "Appraisal-Report-address-appraiser.txt"
template_filename = "story-prompt-with-ontology-v3-template.txt"
output_name = "Appraisal-Report-address-appraiser"
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

In [4]:
bedrock_runtime= boto3.client("bedrock-runtime",region_name="us-east-1")

In [5]:
environment = Environment(loader=FileSystemLoader(template_dir))
template = environment.get_template(template_filename)

In [6]:
with open(os.path.join(ontology_dir,ontology_name),"r") as f:
    ontology = f.read()

In [7]:
with open(os.path.join(input_text_dir,input_text_name),"r") as f:
    input_text = f.read()

In [8]:
prompt = template.render(
    ontology=ontology,
    input_text=input_text
)

In [9]:
prompt

'You are a faithful and trusted network graph maker tasked with creating a comprehensive and interconnected representation of a given context provided between the <input> and </input> tags below\nensuring all nodes are part of a single cluster using the turtle ontology provided between the <ontology> and </ontology> tags. \nYour job  involves to reason step by step and extract key concepts (nodes) and their relationships (links) \nfrom the provided text and constructing a JSON.\n\nWhile doing so, follow these nine guidelines below faithfully:\n\n1. Nodes Identification: Identify key nodes as defined in the classes and attributes section of the ontology\n\n2. Relationship Mapping: Establish clear relationships between all nodes using the definitions provided in the relationships section of the ontology.\n\n3. Relationship Validation: Confirm each relationship\'s validity and relevance based on the text. Avoid introducing external concepts not present in the context.\n\n4. Source and Tar

In [10]:
message={"role": "user","content": prompt}

In [11]:
messages = [message]

In [16]:
body=json.dumps(
{
    'messages':messages,
    'anthropic_version':'bedrock-2023-05-31',
    'top_p':0.5,
    'temperature':0.5,
    'top_k':5,
    'max_tokens':3000
})

In [17]:
response = bedrock_runtime.invoke_model(
    modelId=model_id,
    contentType='application/json',
    accept='application/json',
    body=body
)

In [18]:
response_body = json.loads(response['body'].read())

In [19]:
response_body

{'id': 'msg_01UBywHZ55xSyvY8TJeeAA3i',
 'type': 'message',
 'role': 'assistant',
 'content': [{'type': 'text',
   'text': '```json\n{\n    "nodes": [\n        {\n            "id": "Knowledge",\n            "attributes": "",\n            "type": "concept",\n            "group": 1\n        },\n        {\n            "id": "PropertyAppraisal",\n            "attributes": "hasTitle, hasDate, hasAppraiser, hasRemarks, hasAppraisedValue",\n            "type": "document",\n            "group": 2\n        },\n        {\n            "id": "PropertyAppraisalDate",\n            "attributes": "hasMonth, hasDay, hasYear",\n            "type": "entity",\n            "group": 3\n        },\n        {\n            "id": "PropertyAppraisalMonth",\n            "attributes": "",\n            "type": "entity",\n            "group": 3\n        },\n        {\n            "id": "PropertyAppraisalDay",\n            "attributes": "",\n            "type": "entity",\n            "group": 3\n        },\n        {\

In [23]:
full_output_name = f"{model_id}-{template_filename.split('.txt')[0]}-{output_name}"
full_output_name=full_output_name.replace(":","-")
same_name_count = int(len([f for f in os.listdir(output_dir) if full_output_name in f])/2)
file_count = "" if same_name_count ==0 else f'-{same_name_count}'

In [24]:
full_output_name

'anthropic.claude-3-sonnet-20240229-v1-0-story-prompt-with-ontology-v3-template-Appraisal-Report-address-appraiser'

In [25]:
with open(os.path.join(output_dir,f"{full_output_name}{file_count}.json"),"w+") as f:
    json.dump(response_body,f)

In [26]:
with open(os.path.join(output_dir,f"{full_output_name}-graph{file_count}.json"),"w+") as f:
    json.dump(json.loads(response_body["content"][0]["text"].split("```")[1].split("json")[1]),f)